In [5]:
nRec = 0

with open("offline.final.trace.txt","r") as offline:
    records = offline.readlines()

# Book reports 151,392 records
print(len(records))

for record in records:
    if (record.strip()[0] != "#") :
        nRec += 1
        
# Book reports 146,080 data records
print(nRec)

# Book reports 5312 non-data records
print(len(records) - nRec)

151392
146080
5312


In [12]:
print(records[3].strip().split(";"))
print(len(records[3].strip().split(";")))

['t=1139643118358', 'id=00:02:2D:21:0F:33', 'pos=0.0,0.0,0.0', 'degree=0.0', '00:14:bf:b1:97:8a=-38,2437000000,3', '00:14:bf:b1:97:90=-56,2427000000,3', '00:0f:a3:39:e1:c0=-53,2462000000,3', '00:14:bf:b1:97:8d=-65,2442000000,3', '00:14:bf:b1:97:81=-65,2422000000,3', '00:14:bf:3b:c7:c6=-66,2432000000,3', '00:0f:a3:39:dd:cd=-75,2412000000,3', '00:0f:a3:39:e0:4b=-78,2462000000,3', '00:0f:a3:39:e2:10=-87,2437000000,3', '02:64:fb:68:52:e6=-88,2447000000,1', '02:00:42:55:31:00=-84,2457000000,1']
15


In [28]:
truth = {}
perception = []
p_index = 0

# Loop through the records
for record in records:
    # Ignore the comment lines
    if (record.strip()[0] != "#"):
        # Split the data records on semi-colon
        data_elements = record.split(";")
        num_items = len(data_elements)
        '''
        [0] = t = timestamp
        [1] = id = MAC address of target device
        [2] = pos = comma-delimited X,Y,Z location of target device
        [3] = degree = orientation of target device
        [4]-[n] = Perception data from various access points and ad hoc devices
        
        Fields 0-3 are truth data, 4-n are perception data
        '''
        
        # Parse truth values from fields
        timestamp = float(data_elements[0].split("=")[1])
        tgt_id = data_elements[1].split("=")[1]
        pos_data = data_elements[2].split("=")[1].split(",")
        pos_x = float(pos_data[0])
        pos_y = float(pos_data[1])
        pos_z = float(pos_data[2])
        orient = float(data_elements[3].split("=")[1])
        
        # Create a list of truth parameters
        t_params = [timestamp, tgt_id, pos_x, pos_y, pos_z, orient]
        
        # Add this data to the truth object, referenced by timestamp (so we can search by timestamp)
        truth[timestamp] = t_params
        
        # The rest are perception data, so we loop through those and add individual records for each measurement
        '''
        Perception data is in the form: MAC=amplitude,freq,type
        Split on '=' to get
         [0] : ID
         [1] : measurements
        '''
        if (num_items > 4):
            for iMeas in range(4,num_items):
                p_parts = data_elements[iMeas].split("=")
                sensor_id = p_parts[0]
                measurements = p_parts[1].strip().split(",")
                
                '''
                measurements:
                 [0] = amplitude
                 [1] = frequency
                 [2] = type
                '''
                # Create a list of perception data
                p_params = [float(timestamp), sensor_id, float(measurements[0]),float(measurements[1]),int(measurements[2])]
                
                # Add this data to the perception object referenced by simple index
                perception.append(p_params)
                p_index += 1
                
# Book reports 1,181,628 measurements
print(p_index)

# Last timestamp in the data is 1141936870456
print(truth[1141936870456])

# Last perception data in first record: 02:00:42:55:31:00=-84,2457000000,1
print(perception[10])

# Last perception data in the file: 02:b7:00:bb:a9:35=-72,2447000000,1
print(perception[p_index-1])

1181628
[1141936870456.0, '00:02:2D:21:0F:33', 20.0, 8.0, 0.0, 315.7]
[1139643118358.0, '02:00:42:55:31:00', -84.0, 2457000000.0, 1]
[1141936870456.0, '02:b7:00:bb:a9:35', -72.0, 2447000000.0, 1]
